## LIME Test 1

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
PK_data = pd.read_csv("parkinsons.csv")
PK_data.head()

,name,status,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,...,0.02971,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,...,0.04368,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,1,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,...,0.03590,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,1,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,...,0.03772,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,1,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,...,0.04465,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [3]:
features = ["MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)", "MDVP:Jitter(%)", "MDVP:Jitter(Abs)", "MDVP:RAP", \
            "MDVP:PPQ", "Jitter:DDP", "MDVP:Shimmer", "MDVP:Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5", \
            "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR", "RPDE", "DFA", "spread1", "spread2", "D2", "PPE"]

#features_subset = np.random.choice(features, size=6)
features_subset = features

y = PK_data["status"]

X = PK_data[features_subset]
X.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),...,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,...,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,...,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,...,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,...,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,...,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
# Splitting X & y into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

print('Train Test Shapes:', X_train.shape, X_test.shape)

Train Test Shapes: (156, 22) (39, 22)


In [5]:
#Train and Test a Random Forest with the data
from sklearn.ensemble import RandomForestClassifier

PK_Model = RandomForestClassifier(n_estimators=100, criterion='log_loss')

PK_Model.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss')

In [6]:
print('Model Score', PK_Model.score(X_test, y_test))

Model Score 0.9487179487179487


In [7]:
print('Features', features_subset)
print('Feature Importances', PK_Model.feature_importances_)

Features ['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']
Feature Importances [0.08455533 0.05646223 0.046159   0.0222223  0.0275795  0.03646703
 0.02631561 0.03731993 0.02729713 0.02256251 0.03149081 0.03243985
 0.05251558 0.02762811 0.03388325 0.02455343 0.02014646 0.02569057
 0.13243822 0.06210578 0.05695769 0.11320966]


In [8]:
# LIME Explanation
from lime import lime_tabular

PK_Explain = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train),
                                               mode="classification",
                                               training_labels=None,
                                               feature_names=features_subset)
                                               #categorical_features=None,
                                               #categorical_names=None,
                                               #kernel_width=None,
                                               #kernel=None,
                                               #verbose=False,
                                               #class_names=None,
                                               #feature_selection='auto',
                                               #discretize_continuous=True,
                                               #discretizer='quartile',
                                               #sample_around_instance=False,
                                               #random_state=None,
                                               #training_data_stats=None


In [ ]:
# Looks at Explanation

inst=0

PK_explain_inst = PK_Explain.explain_instance(data_row=X_test.iloc[inst],
                                              predict_fn=PK_Model.predict_proba,
                                              #labels=features_subset)
                                              #top_labels=None,
                                              num_features=5)
                                              #num_samples=5000,
                                              #distance_metric='euclidean',
                                              #model_regressor=None,
                                              #sampling_method='gaussian')

                            
PK_explain_inst.show_in_notebook(#labels=features_subset,
                                 predict_proba=True,
                                 show_predicted_value=True)  #,
                                 #**kwargs)

In [ ]:
print('y_test', y_test.iloc[inst])

In [ ]:
PK_explain_inst.as_list()